# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um die Analyse zu starten, folgen Sie bitte diesen Schritten:

1. **Umgebung vorbereiten:** Gehen Sie oben im Menü auf **Laufzeit** (Runtime) → **Alle ausführen** (Run all).
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.
3. **Berechnung starten:** Sobald das Bedienfeld erscheint, klicken Sie unten auf die grüne Schaltfläche **▶ Ausführen**.

---
*Die Warnung ist eine Google-Sicherheitsmaßnahme für GitHub-Dateien. Die grüne Schaltfläche trainiert das KI-Modell und sendet die Prognosen an die App.*

In [ ]:
# @title 🎮 START CONTROL PANEL
import ipywidgets as widgets
from IPython.display import display, HTML

# UI definieren
style = """<style>.lotto-box{background:#f8f9fa;padding:15px;border-radius:10px;border:1px solid #ddd;}.lbl{font-weight:bold;font-size:16px;margin-bottom:10px;display:block;}</style>"""
display(HTML(style))

lbl = widgets.HTML('<span class="lbl">🎲 LOTTO 6aus49 KI-Generator</span>')
txt_uid = widgets.Text(placeholder="User ID (Auto)", description="🆔 ID:", layout=widgets.Layout(width='200px'))
btn_go = widgets.Button(description="Start Pipeline", button_style='success', icon='rocket', layout=widgets.Layout(width='150px'))
out_log = widgets.Output(layout={'border':'1px solid #ccc','height':'150px','overflow':'auto','margin_top':'10px'})
progress = widgets.IntProgress(value=0, max=100, layout=widgets.Layout(width='100%'))

# Logik-Verbindung ("Bridge" variables for next cell)
global_ui = {
    "btn": btn_go, "txt": txt_uid, "log": out_log, "bar": progress
}

def on_click_wrapper(b):
    # Diese Funktion wird später von Zelle 2 überschrieben/genutzt
    if 'run_lotto_logic' in globals():
        globals()['run_lotto_logic'](global_ui)
    else:
        with out_log: print("⚠️ Bitte Zelle 2 (Code) auch ausführen!")

btn_go.on_click(on_click_wrapper)

# Anzeigen
display(widgets.VBox([lbl, widgets.HBox([txt_uid, btn_go]), progress, out_log])) # .add_class('lotto-box')

# Auto-ID Check (JS Hack)
from google.colab import output
js = """(function(){
  var h=window.location.hash.slice(1);
  var p={}; h.split('&').forEach(function(v){var i=v.split('=');p[i[0]]=i[1];});
  return p.userId || "";
})();"""
try:
    uid_found = output.eval_js(js)
    if uid_found: txt_uid.value = str(uid_found)
except: pass

In [ ]:
# @title ⚙️ CORE SYSTEM (Original Power Model)
import time, requests, json, zipfile, io, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback

BACKEND = "https://apilotto.euroceiling39.ru"

# --- ORIIGNAL POWER MODEL ---
def create_model(input_shape, learning_rate=0.001):
    model = Sequential()
    # Layer 1
    model.add(Bidirectional(LSTM(240, input_shape=input_shape, return_sequences=True)))
    model.add(Dropout(0.2))
    # Layer 2
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Dropout(0.2))
    # Layer 3
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    # Layer 4
    model.add(Bidirectional(LSTM(240, return_sequences=False)))
    model.add(Dropout(0.2))

    # Output Layers
    model.add(Dense(49))              # Intermediate Dense
    model.add(Dense(input_shape[1]))   # Final Output (7 numbers: 6 + Super)

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["accuracy"])
    return model

def run_lotto_logic(ui):
    btn, txt, log_out, bar = ui['btn'], ui['txt'], ui['log'], ui['bar']

    btn.disabled = True
    bar.value = 0
    log_out.clear_output()

    def log(m):
        with log_out: print(f"[{time.strftime('%H:%M:%S')}] {m}")

    uid = txt.value.strip()
    if not uid:
        log("❌ FEHLER: Keine User ID!"); btn.disabled = False; return

    try:
        log(f"🚀 Starte Heavy Analyse für: {uid}")
        bar.value = 5

        # 1. Daten laden
        log("📥 Lade 10 Jahre Archivdaten...")
        r = requests.get("https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip", timeout=30)
        with zipfile.ZipFile(io.BytesIO(r.content)) as z: z.extractall("/content/lotto_temp")

        rows=[]
        with open("/content/lotto_temp/lotto.txt", "r", encoding="utf-8", errors="ignore") as f:
            for l in f.readlines()[1:]:
                p = l.split()
                if len(p)>10 and int(p[2])>=2013: rows.append(p[3:10])

        df = pd.DataFrame(rows).apply(pd.to_numeric, errors='coerce').dropna()
        vals = df.values; vals[:,:6] = np.sort(vals[:,:6], axis=1) # Sort 6 main numbers

        # 2. Preprocessing
        bar.value = 15
        log("⚙️ Preprocessing & Windowing (Win=24)...")

        scaler = StandardScaler().fit(vals)
        scaled = scaler.transform(vals)
        win = 24
        X = np.array([scaled[i:i+win] for i in range(len(scaled)-win)])
        y = np.array([scaled[i+win] for i in range(len(scaled)-win)])

        # 3. Model Build
        log("🏗️ Baue Deep Learning Architektur (4x Bidirectional LSTM)...")
        input_shape = (win, 7)
        model = create_model(input_shape)

        # 4. Training (Heavy)
        EPOCHS = 123
        BATCH = 98

        class P(Callback):
            def on_epoch_end(s,e,l):
                pct = int(20 + 75 * ((e+1)/EPOCHS))
                bar.value = pct
                if e % 20 == 0: log(f"   Epoch {e+1}/{EPOCHS} loss={l['loss']:.4f}")

        log(f"🧠 Starte Training ({EPOCHS} Epochen)... Geduld bitte.")
        model.fit(X, y, epochs=EPOCHS, batch_size=BATCH, verbose=0, callbacks=[P()])

        # 5. Prognose
        log("🔮 Berechne Vorhersage...")
        last_window = scaled[-win:].reshape(1, win, 7)
        pred_scaled = model.predict(last_window)
        pred = scaler.inverse_transform(pred_scaled)[0]

        # Post-Processing (Runden & Bounds)
        nums = [int(round(x)) for x in pred[:6]]
        nums = sorted([min(49, max(1, x)) for x in nums]) # 1-49
        sz = min(9, max(0, int(round(pred[6]))))   # 0-9

        log(f"✅ ERGEBNIS: {nums} [Super: {sz}]")
        bar.value = 98

        # 6. Senden
        log(f"📡 Upload zu App...")
        final_nums = nums

        payload = {
            "userId": uid,
            "numbers": final_nums,
            "generated_at": int(time.time()),
            "model": "LSTM-Deep-v1"
        }
        res = requests.post(f"{BACKEND}/api/colab/update", json=payload)

        if res.ok:
            log("🎉 ERFOLG! Deine Zahlen sind in der App.")
            bar.value = 100
        else:
            log(f"⚠️ Server Fehler: {res.status_code}")

    except Exception as e:
        log(f"💥 CRITICAL ERROR: {e}")

    btn.disabled = False